In [ ]:
import syft as sy
duet = sy.launch_duet(loopback=True)

In [ ]:
import os
import pathlib
current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
outcome_name = "client1"
output_data_dir = pathlib.Path('{}/data/'.format(parent_dir))
pathlib.Path.mkdir(output_data_dir, mode=0o777, parents=True, exist_ok=True)
output_dir = pathlib.Path('{}/result/{}'.format(parent_dir, outcome_name))
pathlib.Path.mkdir(output_dir, mode=0o777, parents=True, exist_ok=True)

In [ ]:
import sys
sys.path.append("..")
import numpy as np
import json
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset # 텐서데이터셋

import random
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

from model.model import attbilstm
from util.communications import *
from util.evaluate_metrics import *


In [ ]:
with open('config.json', 'r') as fo:
    config = json.load(fo)
print(config)

# x_train = np.load("{}/x_train.npy".format(output_data_dir))
# y_train = np.load("{}/y_train.npy".format(output_data_dir))
x_train1 = np.load("{}/x_train1.npy".format(output_data_dir))
y_train1 = np.load("{}/y_train1.npy".format(output_data_dir))
# x_train2 = np.load("{}/x_train2.npy".format(output_data_dir))
# y_train2 = np.load("{}/y_train2.npy".format(output_data_dir))
# x_train3 = np.load("{}/x_train3.npy".format(output_data_dir))
# y_train3 = np.load("{}/y_train3.npy".format(output_data_dir))
x_test = np.load("{}/x_test.npy".format(output_data_dir))
y_test = np.load("{}/y_test.npy".format(output_data_dir))

bsize = config["batch_size"]
gpu = config["gpu"]
device = torch.device('cuda' if gpu and torch.cuda.is_available() else 'cpu')

'''
for b in train_it:
    print (b.text, b.label)
    sys.exit()
'''
def accuracy(pred, label):
    prob, idx = torch.max(pred,1)
    precision = (idx==label).float().mean()
    if gpu:
        accuracy.append(precision.data.item())
    else:
        accuracy.append(precision.data.numpy()[0])
    return np.mean(accuracy)

def cat_accuracy(pred, label):
    max_preds = pred.argmax(dim=1, keepdim=True)
    correct = max_preds.squeeze(1).eq(label)
    #correct = torch.LongTensor([0,4,0,0]).to(device).eq(label)
    correct = correct.sum().unsqueeze(0)
    bs = torch.LongTensor([label.shape[0]]).to(device)
    acc = correct.item() / bs.item()
    #return correct.sum()/torch.LongTensor([label.shape[0]])
    return acc

def train(model, it, lossf, optimizer):
    model.train()
    ep_loss = 0.0
    ep_acc = 0.0
    #loss_values = []
    for x, y in tqdm(it):
        optimizer.zero_grad()
        seq, label = x, y
        pred = model(seq)
        loss = lossf(pred, label)
        acc = cat_accuracy(pred, label)
        loss.backward()
        optimizer.step()
        ep_loss += loss.item()
        ep_acc += acc
        #loss_values.append(ep_loss)
    return ep_loss/ len(it), ep_acc/ len(it)

def evaluate(model, it, lossf):
    model.eval()
    ep_loss = 0.0
    ep_acc = 0.0
    #loss_values = []
    with torch.no_grad():
        for x, y in it:
            seq, label = x, y
            pred = model(seq)
            loss = lossf(pred, label)
            acc = cat_accuracy(pred, label)
            ep_loss += loss.item()
            ep_acc += acc
            #loss_values.append(ep_loss)
    return ep_loss/ len(it), ep_acc/ len(it)

dataset = TensorDataset(torch.tensor(x_train1), torch.LongTensor(y_train1))
train_it = DataLoader(dataset, batch_size=config['batch_size'], shuffle=True)

dataset = TensorDataset(torch.tensor(x_test), torch.LongTensor(y_test))
test_it = DataLoader(dataset, batch_size=config['batch_size'], shuffle=True)

model = attbilstm(config, vec=None)
#lossf = nn.BCEWithLogitsLoss()
lossf = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

if gpu:
    model.to(device)

hist_dict = {}
hist_dict['train_acc'] = []
hist_dict['train_loss'] = []
hist_dict['valid_acc'] = []
hist_dict['valid_loss'] = []

for ep in range(config['epochs']):
    # print ('training epoch...', ep)
    tr_loss, tr_acc = train(model, train_it, lossf, optimizer)
    # print ('TRAIN: loss %.2f acc %.1f' % (tr_loss, tr_acc))
    vl_loss, vl_acc = evaluate(model, test_it, lossf)
    # print('VALID: loss %.2f acc %.1f' % (vl_loss, vl_acc))
    hist_dict['train_acc'].append(tr_acc)
    hist_dict['train_loss'].append(tr_loss)
    hist_dict['valid_acc'].append(vl_acc)
    hist_dict['valid_loss'].append(vl_loss)

te_loss, te_acc = evaluate(model, test_it, lossf)
print('TEST: loss %.2f acc %.1f' % (te_loss, te_acc))


In [ ]:
y_true, y_pred, y_pred_proba = getPredict(model, test_it)
draw_history_plot(hist_dict, output_dir, outcome_name)
draw_confusion_matrix(y_true, y_pred, output_dir, outcome_name)
draw_ROC_AUC(y_true, y_pred_proba, output_dir, outcome_name)
model_performance_evaluation(y_true, y_pred, y_pred_proba, output_dir, outcome_name)

In [ ]:
data_ptrs = sendAllData(model.state_dict(), duet)

In [ ]:
avg_update = getParamWeight(duet.store, request_block=False, delete_obj=True)
model.load_state_dict(avg_update)

In [ ]:
y_true, y_pred, y_pred_proba = getPredict(model, test_it)
draw_history_plot(hist_dict, output_dir, outcome_name)
draw_confusion_matrix(y_true, y_pred, output_dir, outcome_name)
draw_ROC_AUC(y_true, y_pred_proba, output_dir, outcome_name)
model_performance_evaluation(y_true, y_pred, y_pred_proba, output_dir, outcome_name)